In [1]:
from kafka import KafkaConsumer
import json
import pandas as pd
from pymongo import MongoClient
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import udf, col, lower, regexp_replace
from pyspark.ml.feature import Tokenizer, StopWordsRemover, NGram, HashingTF, IDF
from pyspark.ml import Pipeline
from pyspark.sql.types import StructType,StructField
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import LogisticRegressionModel

import nltk
from nltk.corpus import stopwords

# Import dans Pyspark

In [2]:


# Ouverture de la Session Spark
spark = SparkSession.\
builder.\
appName("myApp").\
config("spark.mongodb.input.uri", "mongodb://mongo:27017/db_Booking.*").\
config("spark.mongodb.output.uri", "mongodb://mongo:27017/db_Booking.*").\
config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").\
getOrCreate()



In [3]:
db = spark.read.format("mongo").option("uri","mongodb://mongo:27017/Booking.Review").load()

In [4]:
db.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- hotel_name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- polarity: long (nullable = true)
 |-- prediction: double (nullable = true)
 |-- review: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- word_counts: long (nullable = true)



# Import avec Pandas

In [17]:
# Connection à MongoDB
client = MongoClient('mongo',27017)

#Création de la tables
Booking = client.Booking

#Création de post 
Review = Booking.Review

In [18]:
next(Booking.Review.find())

{'_id': ObjectId('624ed7f0a66bb930cc7cacbc'),
 'hotel_name': 'The Gore Hotel Starhotels Collezione',
 'lat': 51.5001979,
 'long': -0.1795357,
 'review': ' The room we were allocated been coming to this hotel for over 2 decades but when I am leaning over the bath to get hot water to wash my hands because it takes so long for hot water to reach tap in basin it s time to quit coming here beyond quirky now tired ',
 'word_counts': 56,
 'polarity': 0,
 'prediction': 0.0,
 'tags': "[' Leisure trip ', ' Couple ', ' Superior Double Room ', ' Stayed 1 night ', ' Submitted from a mobile device ']"}

In [19]:
cursor= Review.find()
data = list(cursor)
data[:2]

[{'_id': ObjectId('624ed7f0a66bb930cc7cacbc'),
  'hotel_name': 'The Gore Hotel Starhotels Collezione',
  'lat': 51.5001979,
  'long': -0.1795357,
  'review': ' The room we were allocated been coming to this hotel for over 2 decades but when I am leaning over the bath to get hot water to wash my hands because it takes so long for hot water to reach tap in basin it s time to quit coming here beyond quirky now tired ',
  'word_counts': 56,
  'polarity': 0,
  'prediction': 0.0,
  'tags': "[' Leisure trip ', ' Couple ', ' Superior Double Room ', ' Stayed 1 night ', ' Submitted from a mobile device ']"},
 {'_id': ObjectId('624ed7f5a66bb930cc7cacbd'),
  'hotel_name': 'The Principal London',
  'lat': 51.5226217,
  'long': -0.1251602,
  'review': ' The room was cold the heating was set to low ',
  'word_counts': 12,
  'polarity': 0,
  'prediction': 0.0,
  'tags': "[' Business trip ', ' Solo traveler ', ' Single Room ', ' Stayed 1 night ']"}]

In [20]:
df = pd.DataFrame(data)
df.head(2)

,_id,hotel_name,lat,long,review,word_counts,polarity,prediction,tags
0,624ed7f0a66bb930cc7cacbc,The Gore Hotel Starhotels Collezione,51.500198,-0.179536,The room we were allocated been coming to thi...,56,0,0.0,"[' Leisure trip ', ' Couple ', ' Superior Doub..."
1,624ed7f5a66bb930cc7cacbd,The Principal London,51.522622,-0.125160,The room was cold the heating was set to low,12,0,0.0,"[' Business trip ', ' Solo traveler ', ' Singl..."


In [21]:
from sklearn.metrics import balanced_accuracy_score, make_scorer, confusion_matrix, classification_report, average_precision_score,precision_score


conf_matrix_LR=confusion_matrix(df['polarity'],df['prediction'])
conf_matrix_LR = pd.DataFrame(conf_matrix_LR)
conf_matrix_LR

,0,1
0,213,23
1,39,224


In [22]:
cr_RL = classification_report(df['polarity'],df['prediction'])
print(cr_RL)

              precision    recall  f1-score   support

           0       0.85      0.90      0.87       236
           1       0.91      0.85      0.88       263

    accuracy                           0.88       499
   macro avg       0.88      0.88      0.88       499
weighted avg       0.88      0.88      0.88       499

